In [1]:
import csv

INPUT_CSV = 'data_labeled.csv'

CSV_COLUMNS = [
    'id',
    'url',
    'title',
    'img_url',
    'score',
    'servings',
    'prep_time',
    'rating',
    'reviews',
    'made_it_count',
    'calories',
    'total_fat',
    'saturated_fat',
    'cholesterol',
    'sodium',
    'potassium',
    'carbs',
    'dietary_fiber',
    'protein',
    'sugars',
    'vitamin_a',
    'vitamin_c',
    'calcium',
    'iron',
    'thiamin',
    'niacin',
    'vitamin_b6',
    'magnesium',
    'folate',
    # Everything above this index is a <contains> relationship
]

CSV_COLUMN_TYPES = dict({
    'id': 'string',
    'url': 'string',
    'title': 'string',
    'img_url': 'string',
    'score': 'int32',
    'servings': 'int32',
    'prep_time': 'int32',
    'rating': 'float32',
    'reviews': 'int32',
    'made_it_count': 'int32',
    'calories': 'int32',
    'total_fat': 'float32',
    'saturated_fat': 'float32',
    'cholesterol': 'float32',
    'sodium': 'float32',
    'potassium': 'float32',
    'carbs': 'float32',
    'dietary_fiber': 'float32',
    'protein': 'float32',
    'sugars': 'float32',
    'vitamin_a': 'float32',
    'vitamin_c': 'float32',
    'calcium': 'float32',
    'iron': 'float32',
    'thiamin': 'float32',
    'niacin': 'float32',
    'vitamin_b6': 'float32',
    'magnesium': 'float32',
    'folate': 'float32',
    # Everything above this index is a <contains> relationship
})

def parse_data(data_csv, max_rows=None):
    with open(data_csv, 'r') as f:
        reader = csv.reader(f, delimiter=',', quoting=csv.QUOTE_ALL)
        
        col_len = len(CSV_COLUMNS)
        row_count = 0
        
        for row in reader:
            row_arr = [col.replace(r'"', r'\"') for col in row]
            yield row_arr[:col_len]
            row_count += 1
            
            if max_rows is not None and row_count >= max_rows:
                break
            

In [2]:
import pandas as pd

MAX_ROWS = 831
df = pd.DataFrame(data=parse_data(INPUT_CSV, max_rows=MAX_ROWS), columns=(CSV_COLUMNS))

In [3]:
BASE_FEATURES = [
    'calories',
    'total_fat',
    'saturated_fat',
    'cholesterol',
    'sodium',
    'potassium',
    'carbs',
    'dietary_fiber',
    'protein',
    'sugars',
    'vitamin_a',
    'vitamin_c',
    'calcium',
    'iron',
    'thiamin',
    'niacin',
    'vitamin_b6',
    'magnesium',
    'folate',
]

RATE_FEATURES = [
    'rating',
    'reviews',
    'made_it_count',
]

# Augmented columns?
# Calorie percentage from protein
# Calorie percentage from fat
# Calorie percentage from carbs

AUG_FEATURES = [
    'cal_protein',
    'cal_fat',
    'cal_carbs'
]

def compute_aug_cols(row):
    protein_cals = row['protein'] * 4.0
    fat_cals = row['total_fat'] * 9.0
    carb_cals = row['carbs'] * 4.0
    
    total_cals = sum([protein_cals, fat_cals, carb_cals])
    row['cal_protein'] = protein_cals / total_cals
    row['cal_fat'] = fat_cals / total_cals
    row['cal_carbs'] = carb_cals / total_cals
    
    return row


FEATURES = BASE_FEATURES + AUG_FEATURES

DF_COLUMNS = BASE_FEATURES + ['score']
df = df[DF_COLUMNS]

type_dict = {col:CSV_COLUMN_TYPES[col] for col in DF_COLUMNS}
df = df.astype(type_dict)

df = df[df['score'] >= 0]

df = df.apply(compute_aug_cols, axis=1)
df

,calories,total_fat,saturated_fat,cholesterol,sodium,potassium,carbs,dietary_fiber,protein,sugars,...,iron,thiamin,niacin,vitamin_b6,magnesium,folate,score,cal_protein,cal_fat,cal_carbs
0,500.0,21.700001,16.0,5.0,291.0,368.0,72.500000,2.1,6.900000,61.0,...,1.0,0.0,2.0,0.0,31.0,14.0,43.0,0.053812,0.380776,0.565412
1,308.0,18.900000,10.0,73.0,482.0,214.0,26.100000,1.0,8.400000,3.0,...,5.0,0.0,4.0,0.0,17.0,54.0,33.0,0.109055,0.552093,0.338851
2,339.0,12.300000,2.0,0.0,122.0,117.0,56.200001,4.1,3.400000,35.0,...,1.0,0.0,2.0,0.0,19.0,38.0,36.0,0.038957,0.317101,0.643942
3,275.0,12.200000,7.0,57.0,399.0,185.0,29.400000,1.2,11.800000,1.0,...,2.0,0.0,5.0,0.0,29.0,95.0,41.0,0.171886,0.399854,0.428259
4,276.0,15.700000,8.0,73.0,60.0,120.0,31.900000,1.9,4.700000,25.0,...,0.0,0.0,1.0,0.0,4.0,7.0,18.0,0.065346,0.491137,0.443518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,351.0,20.400000,7.0,46.0,1590.0,818.0,15.500000,2.5,27.100000,8.0,...,3.0,0.0,8.0,1.0,32.0,32.0,45.0,0.306215,0.518644,0.175141
825,153.0,9.400000,5.0,29.0,529.0,222.0,10.100000,1.1,7.500000,3.0,...,1.0,0.0,2.0,0.0,19.0,25.0,38.0,0.193548,0.545806,0.260645
826,935.0,58.200001,21.0,161.0,2274.0,822.0,51.500000,4.2,51.099998,6.0,...,9.0,2.0,23.0,1.0,92.0,197.0,54.0,0.218797,0.560694,0.220510
827,508.0,35.099998,13.0,97.0,1096.0,997.0,25.500000,5.6,24.600000,10.0,...,4.0,0.0,10.0,0.0,30.0,20.0,52.0,0.190587,0.611854,0.197560


In [4]:
import numpy as np

X = df[FEATURES].to_numpy()
y = df['score'].to_numpy()

# normalize features
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# normalize labels
mean_y = np.mean(y)
std_y = np.std(y)
y = (y - np.mean(y)) / np.std(y)

print(X.shape, y.shape)

(690, 22) (690,)


In [5]:
import numpy as np

def train_val_test_split(data, train_frac, val_frac, test_frac):
    assert(train_frac + val_frac + test_frac == 1)
    M = data.shape[0]
    np.random.seed(628)
    np.random.shuffle(data)
    return np.split(data, [int(train_frac*M), int((train_frac+val_frac)*M)])

# We use a 3:1:1 split for test train and validation
X_train, X_val, X_test = train_val_test_split(X, 0.6, 0.2, 0.2)
y_train, y_val, y_test = train_val_test_split(y, 0.6, 0.2, 0.2)

X_label = np.concatenate((X_train, X_val), axis=0)
y_label = np.concatenate((y_train, y_val), axis=0)

print(X_train.shape, X_val.shape, X_test.shape)

(414, 22) (138, 22) (138, 22)


In [6]:
# The risk is the average absolute difference
# between the predicted and true nutritional score
def risk(model, X_, y_):
    M_1 = 1.0 / X_.shape[0]
    y_hat = model.predict(X_)
    abs_error = np.abs(y_hat - y_) * std_y
    return M_1 * np.sum(abs_error), np.std(abs_error)

In [7]:
from sklearn.model_selection import cross_val_score

# Perform kfold analysis on the data. 
# Returns a list of validation scores for each fold in the data
def kfold_analysis(model, X_, y_, cv=5):
    return cross_val_score(model, X_, y_, cv=cv)

In [8]:
# Find the data samples with the largest absolute error
# We are doing this because the data was manually labeled and may contain errors
def outlier_analysis(model, X_, y_):
    # We will only analyze data we 
    model.fit(X_, y_)
    y_hat = model.predict(X_)
    abs_error = np.abs(y_hat - y_)
    # abs_error, y, calories
    zipped = list(zip(abs_error, y_, X_[:, FEATURES.index('calories')]))
    return sorted(zipped, key=lambda x: x[0], reverse=True)

In [9]:
def merge_dicts(*dicts):
    res = dicts[0].copy()
    for d in dicts[1:]:
        res.update(d)
    return res

def index_divide(l, i):
    for e in l[:i]:
        yield e
    for e in l[i+1:]:
        yield e

def k_hold_split(split, i):
    train = np.concatenate(tuple(index_divide(split, i)), axis=0)
    val = split[i]
    return train, val

def k_hold_validation(model, D_split, y_split, k):
    """Returns the risk of a model based on k-hold validation"""
    K_1 = 1.0 / k
    acc_risk = 0
    
    for i in range(k):
        D_train, D_val = k_hold_split(D_split, i)
        y_train, y_val = k_hold_split(y_split, i)
        
        model.fit(D_train, y_train)
        acc_risk += risk(model, D_val, y_val)[0]
    
    # Return the average risk
    return K_1 * acc_risk
        
# Train the model using coordinate descent with k-hold validation
# If verbose is True, then everytime the algorithm finds better hyperparameters the risk 
# and parameters will be printed to the console.
# Returns the best hyperparameters and risk
def fit_coordinate_descent(model_type, _X_label, _y_label, k=5, parameters=dict(), verbose=True):
    # Split the data into k parts
    D_split = np.array_split(_X_label, k)
    y_split = np.array_split(_y_label, k)
    
    var_params = []
    common_params = dict()
    for p, v in parameters.items():
        if type(v) is list:
            var_params.append(p)
        else:
            common_params[p] = v
    
    # Train with default hyperparameters
    model = model_type(**common_params)
    best_params = common_params # default
    best_risk = k_hold_validation(model, D_split, y_split, k)
    
    if verbose:
        print(best_risk, 'default')

    fixed_params = { p: parameters[p][0] for p in var_params }
    
    # We try and fit each parameter one at a time
    for p_fit in var_params:
        for v in parameters[p_fit]:
            fitting_param = { p_fit: v }
            params = merge_dicts(common_params, fixed_params, fitting_param)
            model = model_type(**params)
            
            # Run a full k-hold validation to test the parameter combination
            iter_risk = k_hold_validation(model, D_split, y_split, k)
            
            if iter_risk < best_risk:
                # If the hyperparameters are better, save it to the fixed parameters list
                fixed_params[p_fit] = v
                best_risk = iter_risk
                best_params = params
                
                if verbose:
                    print(iter_risk, params)
    
    return best_params, best_risk

In [10]:
from sklearn.linear_model import LinearRegression

# Get the top 20 bad samples. Data should be unnormalized before running so that samples can be identified
outlier_analysis(LinearRegression(), X, y)[:20]

[(2.0017854349123425, -1.9938447464282343, -0.6864355850846982),
 (1.6673873072662124, -1.733945083633909, -0.9544029687482163),
 (1.6564258381101458, -1.785925016192774, 0.5440268909212526),
 (1.6277812107628649, 1.6967304652511852, -0.9325280802858883),
 (1.566320958550599, 2.6323692513107564, -1.4684628476129247),
 (1.522436893616936, -1.2141457580452584, -1.3919007379947765),
 (1.5065532673518436, -1.733945083633909, 0.3252780062979725),
 (1.4983944701799077, -0.6423664998977426, 1.079961658248289),
 (1.4640081630306612, 2.5803893187518914, -1.0637774110598563),
 (1.4555627343137114, -1.058205960368663, 2.8627650679280223),
 (1.3775717257671358, -1.5260253533984487, -1.2442452408740625),
 (1.3713121957785013, -1.1101858929275283, -1.2223703524117344),
 (1.36799894594033, -0.2785069719856872, 0.6260577226549827),
 (1.3652452767981518, -0.9022661626920679, -0.47315542257700005),
 (1.354391625712176, 1.6447505326923202, -1.2387765187584805),
 (1.3531114617519056, 1.6447505326923202, -

In [11]:
from sklearn.linear_model import LinearRegression

# Run k-fold cross validation analysis. We expect that the validation score for each fold 
# is roughly equal if the partitions are iid.
print(kfold_analysis(LinearRegression(), X_label, y_label))

[0.64378339 0.64978618 0.62663227 0.5839701  0.60233383]


In [12]:
# Implement trivial estimator (guesses mean) to get upper bound on acceptable error
class TrivialEstimator:
    def __init__(self):
        self.mean = 0
    
    def fit(self, X, y):
        self.mean = np.mean(y)
    
    def predict(self, _):
        return self.mean
    
est = TrivialEstimator()
est.fit(X_train, y_train)
print('Validation Risk', risk(est, X_val, y_val))

Validation Risk (16.816565147378, 10.651524788856511)


In [13]:
from sklearn.linear_model import LinearRegression

best_linreg_params, best_linreg_risk = fit_coordinate_descent(LinearRegression, X_label, y_label)

9.459243994315843 default


In [14]:
from sklearn.linear_model import Ridge

ridge_params = {
    'random_state': 628,
    'max_iter': [None, 100, 1000],
    'alpha': [0.01, 0.3, 1, 3, 10], # Controls strength of l2 regularization
}

best_ridge_params, best_ridge_risk = fit_coordinate_descent(Ridge, X_label, y_label, parameters=ridge_params)

9.509923859903493 default
9.456763772222752 {'random_state': 628, 'max_iter': None, 'alpha': 0.01}


In [15]:
from sklearn.linear_model import ElasticNet

enet_params = {
    'random_state': 628,
    'max_iter': 10000,
    'l1_ratio': [1.0, 0.7, 0.5, 0.3, 0.1, 0.01, 0.001], # controls weighted portion of l1 regularization. 1.0 means full l_1 regularization
    'alpha': [0.001, 0.01, 0.1], # Controls strength of regularization
}

best_enet_params, best_enet_risk = fit_coordinate_descent(ElasticNet, X_label, y_label, parameters=enet_params)

16.315889703788866 default
9.504338282118882 {'random_state': 628, 'max_iter': 10000, 'l1_ratio': 1.0, 'alpha': 0.001}
9.490512726244082 {'random_state': 628, 'max_iter': 10000, 'l1_ratio': 0.7, 'alpha': 0.001}
9.485070810057264 {'random_state': 628, 'max_iter': 10000, 'l1_ratio': 0.5, 'alpha': 0.001}
9.48144124676106 {'random_state': 628, 'max_iter': 10000, 'l1_ratio': 0.3, 'alpha': 0.001}
9.478983627737641 {'random_state': 628, 'max_iter': 10000, 'l1_ratio': 0.1, 'alpha': 0.001}
9.478222175947286 {'random_state': 628, 'max_iter': 10000, 'l1_ratio': 0.01, 'alpha': 0.001}
9.478138604019149 {'random_state': 628, 'max_iter': 10000, 'l1_ratio': 0.001, 'alpha': 0.001}


In [16]:
from sklearn.linear_model import TheilSenRegressor

fit_coordinate_descent(TheilSenRegressor, X_label, y_label)

10.337839937315657 default


({}, 10.337839937315657)

In [17]:
from sklearn.neural_network import MLPRegressor

nn_params = {
    'random_state': 628,
    'verbose': False,
    'max_iter': 1000,
    'solver': ['adam', 'lbfgs', 'sgd'], # adam solver, newton's method or stochastic gradient descent
    'activation': ['relu', 'identity'],
    'hidden_layer_sizes': [(100,), (len(FEATURES),), (len(FEATURES*2),), (100,100), (len(FEATURES*2),len(FEATURES*2))],
    'alpha': [0.0001, 0.001, 0.01, 0.1],
}

best_nn_params, best_nn_risk = fit_coordinate_descent(MLPRegressor, X_label, y_label, parameters=nn_params)

8.217816540046561 default


/mnt/overflow/ece493/nutrition-regressor/.venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/mnt/overflow/ece493/nutrition-regressor/.venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/mnt/overflow/ece493/nutrition-regressor/.venv/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_pe

8.171270699450522 {'random_state': 628, 'verbose': False, 'max_iter': 1000, 'solver': 'sgd', 'activation': 'relu', 'hidden_layer_sizes': (100,), 'alpha': 0.0001}
7.618993571809913 {'random_state': 628, 'verbose': False, 'max_iter': 1000, 'solver': 'sgd', 'activation': 'relu', 'hidden_layer_sizes': (100, 100), 'alpha': 0.0001}


In [18]:
print(best_nn_params)
reg_final = MLPRegressor(**best_nn_params).fit(X_label, y_label)
print('Test Score: ', risk(reg_final, X_test, y_test))

{'random_state': 628, 'verbose': False, 'max_iter': 1000, 'solver': 'sgd', 'activation': 'relu', 'hidden_layer_sizes': (100, 100), 'alpha': 0.0001}
Test Score:  (7.104656565791909, 6.5444618276223165)


In [19]:
import pickle

# Save the model to a file
fname = 'nutriscore.model'
pickle.dump(reg_final, open(fname, 'wb'))